# **BERT(multi-lingual) 기반의 관용구 분류기 (Hugging Face Pytorch ver.)**
Idiom Classifier base BERT(multi lingual) with Hugging Face BERT Pytorch ver.

- 최종 정리 일시: 2021.10.19

- Reference Code: [네이버 영화리뷰 감성분석 (Colab)](https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x&uniqifier=1)

 - 위 코드가 네이버 영화 리뷰를 긍/부정으로 나누는 Binary Classification이기 때문에 관용구를 Yes/No로 분류하는 관용구 분류기와 Dataset만 다를 뿐 기본적으로 모두 구조는 동일
 - 기본적인 전체 코드는 유지한 채, 데이터와 이에 맞춰 일부 파라미터를 변경하여 관용구 분류기를 구현


* KoBERT or KoGPT2의 버전과는 다르게 새로운 데이터셋에 대해 관용구를 예측하는 코드는 수록되어 있지 않음

# **1. 환경 설정**

In [ ]:
# 경로 설정(Colab에서 진행한 경우)
%cd /content/drive/MyDrive/Colab Notebooks/2021_NLP_Project

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/2021_NLP_Project'
/content


## 1-1) 변수 설정

- 일부 파라미터들 중 중복되는 몇 가지 파라미터에 대해 변수로 처리하고자 추가

 - `max_len_value`: 입력 토큰의 최대 시퀀스 길이 (`MAX_LEN`의 입력 값)
 - `bert_model_name`: 분류를 위한 BERT 모델 생성 (`BertForSequenceClassification.from_pretrained`의 입력 값)
 - `train_test_split_ratio`: train/test 분리 비율 (`train_test_split`의 입력 값)
 - `train_val_split_ratio `: 훈련셋과 검증셋으로 분리 (`train_test_split`의 입력 값)
   - train 데이터의 train/validation 분리 비율로 약 5000개:400개로 분리하기 위해 설정
   - 어텐션 마스크를 훈련셋과 검증셋으로 분리 에도 사용
 - `batch_size_value`: 배치 사이즈 (`batch_size`의 입력 값)
 - `learning_rate`: 옵티마이저 설정 (`AdamW`의 입력 값)
   - 처음 default는 2e-5 (관용구 분류기의 경우 이를 변경하여 성능을 끌어올렸음)
 - `epochs_value`: 에폭수 (`epochs`의 입력값)

In [ ]:
max_len_value = 64
bert_model_name = 'bert-base-multilingual-cased'
train_test_split_ratio = 0.2
train_val_split_ratio = 0.07
batch_size_value = 64
learning_rate = 1e-5
epochs_value = 3

## 1-2) Library Import

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
     |████████████████████████████████| 636 kB 28.5 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import pickle
# from tqdm import tqdm

# **2. 전처리 - 훈련셋**

## 2-1) Dataset Load

- Step 2) Dataset
```python
with open('./data/final_idiom_dataset_for_ko.pkl', 'rb') as f:
```
> 한-영 관용구 기계번역을 위한 NMT 학습 방법

    - 위 논문에서 사용한 한국어 관용구 Dataset 3,376개
    - AI Hub의 한-영 번역(병렬) 말뭉치에서 관용구가 없는 문장 임의로 3,376개 추출

- Step 3) Dataset Case 1
```python
with open('./data/train_11710.pkl', 'rb') as f:
```
    - 5,855개(관용구 o) + 5,855개(관용구 x)로 테스트 해보기
    - 검증셋: 1,500개(관용구 o) + 1,500개(관용구 x)

- Step 3) Dataset Case 2
```python
with open('./data/train_34614.pkl', 'rb') as f:
```
    - 17,307(관용구 o) + 17,307(관용구 x)


In [ ]:
# 파일 로드 (경로는 각자 상황에 맞게 설정할 것!)
with open('./data/final_idiom_dataset_for_ko.pkl', 'rb') as f:
  corpus = pickle.load(f)

# with open('./data/train_34614.pkl', 'rb') as f:
#   corpus = pickle.load(f)

In [ ]:
# Dataset을 불러온다.
corpus.head(10)

,ko,en,Label
0,다만 지난 7월부터 수면 무호흡증 진단을 위한 수면다원검사와 치료에 필요한 양압기 ...,"However, since July, as health insurance (20% ...",0
1,하지만 몇몇 사람들은 그 표현이 관용구인지 알아차리지 못합니다.,"Some people, however, cant recognize that it w...",0
2,끈끈한 승부근성으로 찬스에서 더 영양가 만점의 활약을 하는 오재원이 있기에 선두질주...,The Doosan Bears are gaining momentum in the l...,1
3,협약내용은 삼성화재서비스손해사정㈜에서 매월 임직원들의 기부를 통해 향후 3년간 30...,The contents of the agreement are to support m...,0
4,이 같은 대중적 관심을 겨냥해 이동통신사가 발 빠르게 움직였다.,"In response to such public interest, mobile op...",1
5,"달러보험은 원화상품 대비 달러가 갖는 경쟁력을 바탕으로 고객가치를 높인 상품으로, ...",The dollar insurance is a product that heighte...,0
6,극 중 커플로 등장했던 박경림과 조인성이 사석에서 술잔을 기울이 박경림의 절친인 장...,"Park Kyung-rim and Jo In-sung, who appeared as...",1
7,"제1 야당이 극우 인사의 해괴망측한 주장에 멍석을 깔아주고, 함께 장단을 맞췄는데 ...",The main opposition party dug up the weird cla...,1
8,금융규제에 발이 묶여 시장 진입 자체가 힘든 상황이 지속되자 국회와 정부 부처가 모...,As financial regulations continue to make it d...,1
9,1년 이상의 파트 타임 경험이 있습니다.,I have an experience of part time job longer t...,0


In [ ]:
# 문장과 Label 추출
text = corpus['ko']
label = corpus['Label']

# Label 개수 변수 지정 (단순히 2를 값으로 집어 넣어도 됨)
n_topic = len(set(label))

In [ ]:
# 관용구 데이터와 관용구가 아닌 데이터의 비율이 1:1인지 확인
label.sum(), label.sum()/len(label)

(3376, 0.5)

## 2-2) Train/Test/Validation 분할

In [ ]:
# Train 데이터와 Test 데이터 분리 (분리 비율은 `train_test_split_ratio`에서 설정 가능)
x_train, x_test, y_train, y_test = train_test_split(text, label, test_size=train_test_split_ratio, random_state = 42)

## 2-3) Encoding

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(tex) + " [SEP]" for tex in x_train]
sentences[:10]

['[CLS] 나무들이 우리를 돕는 여러 방법이 있습니다. [SEP]',
 '[CLS] 소득주도 경제가 탄력을 받으려면 일자리가 증가할 수 있도록 산업 구조 개선과 연구개발(R&D) 지원이 있어야 한다. [SEP]',
 '[CLS] 특히 자극적인 대중음악의 트렌드는 점점 음악 속에서 ‘사람’을 지웠고, 몸이 먼저 반응하기 시작하며 따뜻했던 음악의 미덕은 설 자리를 잃었다. [SEP]',
 '[CLS] 높은 인건비는 차치하더라도 사사건건 기업 경영의 발목을 잡는 노조도 기업들이 해외로 눈을 돌리는 원인으로 꼽힌다. [SEP]',
 '[CLS] 내가 가지고 있는 소유를 팔아 구제해서 마련해야 합니다. [SEP]',
 '[CLS] 은행권 노사가 공동으로 2000억원대 규모의 공익재단을 만들기로 했다. [SEP]',
 '[CLS] 조금 전에 결제한 카드랑 영수증은 여기에 있어요. [SEP]',
 '[CLS] 교육감들이 교육부 개혁방향과 공통분모를 찾아 손발을 맞춰야 하는 이유다. [SEP]',
 '[CLS] 귀환 후 외가가 있던 경주에서 살다가 아이가 있는 홀아비와 결혼하였으나 남편이 일찍 사망하였다. [SEP]',
 '[CLS] 롯데그룹이 최근 실적 악화로 골치를 앓고 있는 롯데마트의 신임 대표에 문영표(사진) 롯데글로벌로지스 대표를 구원 등판시키며 새로운 활력을 꾀한다. [SEP]']

### ***Referce Code 설명 인용1***
- BERT의 입력은 아래 그림과 같은 형식

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

- Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입
 - 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 수행
- [SEP]은 Seperation을 가리키며 두 문장을 구분하는 역할
 - 본 예측에서는 문장이 하나이므로 [SEP]도 하나만 넣어서 진행
<br>


In [ ]:
# 라벨 추출
labels = y_train.values
labels

array([0, 1, 1, ..., 0, 0, 0])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='bert_kor_ckpt', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] 나무들이 우리를 돕는 여러 방법이 있습니다. [SEP]
['[CLS]', '나', '##무', '##들이', '우', '##리를', '돕', '##는', '여러', '방', '##법', '##이', '있', '##습', '##니다', '.', '[SEP]']


### ***Referce Code 설명 인용2***

- BERT는 형태소분석으로 토큰을 분리하지 않고 WordPiece라는 통계적인 방식을 사용
- 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듦
    - 언어에 상관없이 토큰을 생성할 수 있다는 장점
    - 신조어 같이 사전에 없는 단어 처리 용이

- 위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시
- 토크나이저는 여러 언어의 데이터를 기반으로 만든 bert-base-multilingual-cased'를 사용. → 한글도 처리가 가능
<br>


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = max_len_value

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8982,  32537,  20173,   9604,  27852,   9093,  11018,
        30085,   9328,  33768,  10739,   9647, 119081,  48345,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

### ***Referce Code 설명 인용3***

- 보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없음
- 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용함
- BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있으며, 이를 참조하여 토큰을 인덱스로 변환하는 과정을 수행
<br>


In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=train_val_split_ratio)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=train_val_split_ratio)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8885,  30858,  18623,   8922,  66554,   9711,  88168,   9531,
        118802,  11664,   9323, 101322,  10530,   9368,  10739,   9141,  46572,
          9519,  46861,  16439,  50632,  28000,   9998,  12945,   9776,  10892,
          8996,  14646,   9460, 119081,  10530,   9521,  18784, 119471,  10622,
          9511,  11664,  13767,   9283,  37114, 119031,  11903,    119,    102,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0])
tensor(1)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([  101,  9954,  9485, 36553, 10739,  9484, 27852,  8932, 14279, 37341,
         9854, 63243, 29669, 10017, 27023, 116

In [ ]:
# 배치 사이즈
batch_size = batch_size_value

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# **3. 전처리 - 테스트셋**

### 3-1) Encoding
- 2장 전처리 - 훈련셋 / 2-3) Encoding와 동일한 과정

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in x_test]
# sentences[:10]

In [ ]:
# 라벨 추출
labels = y_test.values
# labels

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
# tokenizer = RobertaTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
# tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[CLS] 김장언 미술 큐레이터를 비롯해 김남수 무용평론가, 영화이론을 전공한 임경용 더북소사이어티 대표 등 다양한 분야의 전문가들이 머리를 맞댔다. [SEP]
['[CLS]', '김', '##장', '##언', '미', '##술', '큐', '##레', '##이터', '##를', '비', '##롯', '##해', '김', '##남', '##수', '무', '##용', '##평', '##론', '##가', ',', '영화', '##이', '##론', '##을', '전', '##공', '##한', '임', '##경', '##용', '더', '##북', '##소', '##사', '##이어', '##티', '대', '##표', '등', '다양한', '분', '##야', '##의', '전', '##문', '##가', '##들이', '머', '##리를', '[UNK]', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = max_len_value

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8935,  13890,  48036,   9309,  51945,   9832,  56645,
        85297,  11513,   9379, 118885,  14523,   8935,  37004,  15891,
         9294,  24974, 119398,  42769,  11287,    117,  42428,  10739,
        42769,  10622,   9665,  28000,  11102,   9644,  31720,  24974,
         9074,  82512,  22333,  12945,  86732,  45725,   9069,  37824,
         9121,  53645,   9367,  21711,  10459,   9665,  25934,  11287,
        20173,   9265,  27852,    100,    119,    102,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8935,  13890,  48036,   9309,  51945,   9832,  56645,  85297,
         11513,   9379, 118885,  14523,   8935,  37004,  15891,   9294,  24974,
        119398,  42769,  11287,    117,  42428,  10739,  42769,  10622,   9665,
         28000,  11102,   9644,  31720,  24974,   9074,  82512,  22333,  12945,
         86732,  45725,   9069,  37824,   9121,  53645,   9367,  21711,  10459,
          9665,  25934,  11287,  20173,   9265,  27852,    100,    119,    102,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0])
tensor(1)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
# 배치 사이즈
batch_size = batch_size_value

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# **4. 모델 생성**

## 4-1) GPU 설정

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 4-2) BERT Model 생성

- 만약 Binary Classification이 아닌 다중 분류일 경우 `BertForSequenceClassification.from_pretrained`의 `num_labels`의 수 변경

In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

### ***Referce Code 설명 인용4***

- 사전훈련된 BERT는 다양한 문제로 ***전이학습***이 가능
- 본 코드에서는 아래 그림과 같이 한 문장을 분류하는 방법을 사용

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

- 관용구 문장이 입력으로 들어가면 긍정(관용구ㅇ)/부정(관용구x)으로 구분
- 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 ***파인튜닝***을 수행
- Huggning Face는 **[BertForSequenceClassification()](https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification)** 함수를 제공하기 때문에 쉽게 구현 가능
    - BertForSequenceClassification 추가 설명 사이트 [↩](https://medium.com/huggingface/multi-label-text-classification-using-bert-the-mighty-transformer-69714fa3fb3d)
<br>


In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = learning_rate, # 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = epochs_value

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# **5. 모델 학습**

## 5-1) 성능 평가 함수 생성
- Accuracy, Precision, Recall, F1 Score를 계산하기 위한 함수

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)
    precision = precision_score(pred_flat, labels_flat)
    recall = recall_score(pred_flat, labels_flat)
    f1score = f1_score(pred_flat, labels_flat)
    return accuracy, precision, recall, f1score

## 5-2) 학습 수행
- 에폭마다 훈련셋과 검증셋을 반복하여 학습을 수행

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy, _, _, _ = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...

  Average training loss: 0.61
  Training epcoh took: 0:01:40

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:03

======== Epoch 2 / 3 ========
Training...

  Average training loss: 0.36
  Training epcoh took: 0:01:40

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:03

======== Epoch 3 / 3 ========
Training...

  Average training loss: 0.25
  Training epcoh took: 0:01:40

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:03

Training complete!


In [ ]:
# 모델 저장
torch.save(model, './model/BERT_multilingual_pytorch.model')

# **6. 테스트셋 평가**
- 2장 전처리 - 훈련셋 / 2-2) Train/Test/Validation의 Test Data에 대한 성능 평가 과정

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1score = 0, 0, 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy, tmp_eval_precision, tmp_eval_recall, tmp_eval_f1score = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    eval_precision += tmp_eval_precision
    eval_recall += tmp_eval_recall
    eval_f1score += tmp_eval_f1score

    nb_eval_steps += 1

print("")
print("Precision: {0:.3f}".format(eval_precision/nb_eval_steps))
print("Recall: {0:.3f}".format(eval_recall/nb_eval_steps))
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("f1_score: {0:.3f}".format(eval_f1score/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Precision: 0.890
Recall: 0.877
Accuracy: 0.885
f1_score: 0.874
Test took: 0:00:09


# **7. 새로운 문장 테스트**

## 7-1) Dataset Load

- 경우 1
```python
new_test = pd.read_csv('./data/new_idiom_dataset(100).csv')
```
    - 뉴스 기사에서 선별한 관용구가 없는 문장 50개
    - 국립국어원 표준국어대사전에 수록된 관용구 예문 50문장 (ㄱ~ㅎ까지 고르게 선별)

- 경우 2
```python
with open('./data/test_3000.pkl', 'rb') as f:
    new_test = pickle.load(f)
```
    - Step 3)의 Dataset Case 1 & 2의 검증셋: 1,500개(관용구 o) + 1,500개(관용구 x)는 모두 동일




In [ ]:
# 데이터 로드
new_test = pd.read_csv('./data/new_idiom_dataset(100).csv')

# with open('./data/test_3000.pkl', 'rb') as f:
#   new_test = pickle.load(f)

In [ ]:
# 데이터 정보 확인
new_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ko      100 non-null    object
 1   Label   100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


## 7-2) BERT Model Load

In [ ]:
# 모델 로드
test_model = torch.load('./model/BERT_multilingual_pytorch.model')
test_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

## 7-3) 전처리 및 Encoding
- 2장 전처리 - 훈련셋 과정과 동일

### 쓰이지 않는듯한 코드들

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = max_len_value

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

### 수행

In [ ]:
nex_text_ko = new_test['ko']
new_text_label = new_test['Label']

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in nex_text_ko]
# sentences[:10]

In [ ]:
# 라벨 추출
labels = new_text_label.values
# labels

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 윤석열 전 총장이 검찰총장으로 재직하던 지난해 총선 직전, 검찰이 국민의힘의 전신인 미래통합당을 통해 여권 정치인들에 대한 고발을 요청했다는 의혹이 제기됐습니다.  [SEP]
['[CLS]', '윤', '##석', '##열', '전', '총', '##장이', '검', '##찰', '##총', '##장으로', '재', '##직', '##하던', '지', '##난', '##해', '총', '##선', '직', '##전', ',', '검', '##찰', '##이', '국', '##민', '##의', '##힘', '##의', '전', '##신', '##인', '미', '##래', '##통', '##합', '##당', '##을', '통해', '여', '##권', '정', '##치', '##인', '##들', '##에', '대한', '고', '##발', '##을', '요', '##청', '##했다', '##는', '의', '##혹', '##이', '제', '##기', '##됐', '##습', '##니다', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = max_len_value

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9627,  40958,  79604,   9665,   9761,  55635,   8868,
        99118, 119270,  66896,   9659,  33077,  76424,   9706,  33305,
        14523,   9761,  18471,   9707,  16617,    117,   8868,  99118,
        10739,   8909,  36553,  10459, 119471,  10459,   9665,  25387,
        12030,   9309,  37388,  43022,  33188,  21928,  10622,  25605,
         9565,  25347,   9670,  18622,  12030,  27023,  10530,  18154,
         8888,  51431,  10622,   9599,  40311,  12490,  11018,   9637,
       119438,  10739,   9672,  12310, 118799, 119081,  48345,    119])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9627,  40958,  79604,   9665,   9761,  55635,   8868,  99118,
        119270,  66896,   9659,  33077,  76424,   9706,  33305,  14523,   9761,
         18471,   9707,  16617,    117,   8868,  99118,  10739,   8909,  36553,
         10459, 119471,  10459,   9665,  25387,  12030,   9309,  37388,  43022,
         33188,  21928,  10622,  25605,   9565,  25347,   9670,  18622,  12030,
         27023,  10530,  18154,   8888,  51431,  10622,   9599,  40311,  12490,
         11018,   9637, 119438,  10739,   9672,  12310, 118799, 119081,  48345,
           119])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [ ]:
# 배치 사이즈
batch_size = batch_size_value

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 7-4) 테스트셋 평가
- 6장 테스트셋 평가 과정과 동일

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1score = 0, 0, 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy, tmp_eval_precision, tmp_eval_recall, tmp_eval_f1score = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    eval_precision += tmp_eval_precision
    eval_recall += tmp_eval_recall
    eval_f1score += tmp_eval_f1score

    nb_eval_steps += 1

print("")
print("Precision: {0:.3f}".format(eval_precision/nb_eval_steps))
print("Recall: {0:.3f}".format(eval_recall/nb_eval_steps))
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("f1_score: {0:.3f}".format(eval_f1score/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Precision: 0.901
Recall: 0.894
Accuracy: 0.896
f1_score: 0.897
Test took: 0:00:01
